In [1]:
import pandas as pd
import numpy as np
import string
from lorem_text import lorem
import random
from random_word import RandomWords
import os

In [2]:
for d in ['data','data/tree_splits']:
    try:
        os.mkdir(d)
    except:
        print(f'Directory "{d}" already exists')

In [3]:
n_mice = 378
n_bacteria = 56
n_normal = 36
n_binary = 20
n_behaviors = 35

In [4]:
t = list(set(lorem.words(n_bacteria*10).lower().replace(',','').replace('.','').replace('\n',' ').split(' ')))
if len(t)>n_bacteria:
    print(f'Generated {len(t)} synthetic words')
    genus_names = [f'g__{g}' for g in t]
else:
    print(f'Not enough synthetic words - t is {len(t)}')

Generated 182 synthetic words


In [5]:
r = RandomWords()
behavior_names = []
for b in range(0,n_behaviors):
    prepend = ''.join(random.sample(list(string.ascii_uppercase),3))
    behavior_names.append(f'{prepend}_{r.get_random_word().title()}')

In [6]:
behaviors = pd.DataFrame(columns=behavior_names)

In [7]:
genus = pd.DataFrame(columns=genus_names[:n_bacteria])

In [8]:
for b in behaviors.columns:
    behaviors[b] = np.random.normal(-2, 2, n_mice)

In [9]:
%%capture
for g_norm in genus_names[:n_normal]:
    genus[g_norm] = list(np.random.normal(-2, 2, n_mice))
for g_bin in genus_names[n_normal:]:
    genus[g_bin] = list(np.random.chisquare(1, n_mice))

In [10]:
behaviors.to_csv('data/behaviors.csv', index=False)
genus.to_csv('data/genus.csv', index=False)

In [19]:
tree_splits = pd.DataFrame(columns=['source','target','split','split_value','nobs','behavior'])
summary = tree_splits.copy()
for b in behavior_names:
    tree = tree_splits.copy()
    n = random.choice(range(5,7))
    tree['source'] = pd.Series(random.sample(genus_names,n))
    tree['target'] = pd.Series(random.sample(genus_names,n))
    tree['split'] = pd.Series(random.choices(['<','>'],k=n))
    tree['split_value'] = pd.Series(np.random.normal(-2, 2, n))
    tree['nobs'] = pd.Series(random.sample(range(1,n_mice),n))
    tree['behavior'] = [b]*n
    tree = tree[tree['source']!=tree['target']]
    tree.to_csv(f'data/tree_splits/{b}.csv', index=False)
    summary = pd.concat([summary,tree])

In [20]:
summary.to_csv('data/summary_tree.csv')